# Load Packages

In [1]:
using MathProgBase, Ipopt, Plots, JLD

include("printmat.jl")

println4Ps

# Loading Data


from a jld file

In [2]:
x = load("Data/Fin1FPerfEvalEmp.jld")

(RFunds,FundNames,Rb,IndNames,dN) = (x["RFunds"],x["FundNames"],x["Rb"],x["IndNames"],x["dN"])

println("\nthe funds: ")
println(FundNames)

println("\nindicies: ")
println(IndNames)


the funds: 
String["Putnam Asset Allocation: Growth A", "Vanguard Wellington"]

indicies: 
String["Equity: US, LargeCap, Value", "Equity: US, LargeCap, Growth", "Equity: US, MidCap", "Equity: US, SmallCap", "Equity: Int. (ex US), Developed", "Equity: Emerging Markets", "Fixed Income: US, Bills", "Fixed Income: US, Gov. Bonds", "Fixed Income: US, Corp. Bonds", "Fixed Income: Non-US, Bonds"]


# A Function for Style Analysis


Notice:  
(a) The regression is $Y = a_1 X_1 + ...+ a_K X_K + u$,
               where $a_j >= 0$, $a_j <= 1$ and $a_1+...+a_K = 1$
               
(b) uses quadprog from MathProgBase, which solves

 min  $0.5 * b'Qb + c'b$
 s.t. $Aeq * x = beq$ and 
 $lb \le x \le ub$
                    
(c) Write the sum of squered residuals as   
$(Y-Xb)'(Y-Xb) = Y'Y - Y'Xb - b'X'Y + b'X'Xb$, which equals
$Y'Y - 2(Y'X)b + b'X'Xb$,
                              
which is proportional to $0.5*b'(X'X/T)b + (-(Y'X/T)')'b$,so  

$Q = X'X/T$ and $c = -(Y'X/T)'$
                              
(d) Here, X = Rm, Y = R

In [3]:
function StyleAnalysisPs(Y,X)

    (T,K) = size(X,1,2)

    Q   = X'X/T
    ub  = ones(K)                         #w <= 1
    lb  = zeros(K)                        #w >= 0
    Aeq   = ones(1,K)                     #1'w = 1
    beq = 1.0

    b_ls = X\Y                          #LS estimate of weights, no restrictions
    c    = -(Y'X/T)'
    Sol  = quadprog(c,Q,Aeq,'=',beq,lb,ub,IpoptSolver(print_level=0))
    if Sol.status == :Optimal
      b = Sol.sol
    else
      b = NaN
    end

  return b,b_ls

end

StyleAnalysisPs (generic function with 1 method)

In [4]:
(b,b_ls) = StyleAnalysisPs(RFunds[:,1],Rb)

println("OLS and style analysis coeffs")
printmat([b_ls b])

printlnPs("sum of coeffs: ",sum([b_ls b],1))

vvSort = sortperm(b)    #indices so that b[vvSort] is sorted in ascending order
printlnPs("\nthe 3 most important benchmarks are ",vvSort[end-2:end])


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

OLS and style analysis coeffs
     0.128     0.129
     0.099     0.099
     0.076     0.075
     0.064     0.066
     0.412     0.412
     0.047     0.049
     0.090     0.062
    -0.066     0.004
     0.090     0.051
     0.095     0.053

sum of coeffs:      1.037     1.000

the 3 most important benchmarks are          2         1         5


# Redo the Style Analysis on a Moving Data Window 

In [5]:
(T,K)     = size(Rb,1,2)
WinSize = 104

b = fill(NaN,(T,K))
for t = (WinSize+1):T
    vv     = (t-WinSize):t
    b_i,   = StyleAnalysisPs(RFunds[vv,1],Rb[vv,:])
    b[t,:] = b_i
end

In [12]:

xTicksLoc = Dates.value.([Date(2000);Date(2005);Date(2010);Date(2015)])
xTicksLab = ["2000";"2005";"2010";"2015"]               #crude way of getting tick marks right

plot(dN,b[:,vvSort[end-2:end]],color=[:blue :red :green],label=IndNames[vvSort[end-2:end]],xticks=(xTicksLoc,xTicksLab))
title!("Loadings on the 3 most important assets over the last $WinSize months")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2000 
 
 
 2005 
 
 
 2010 
 
 
 2015 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 0.6 
 
 
 Loadings on the 3 most important assets over the last 104 months 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 84.326,342.181 84.827,346.818 85.328,346.132 85.829,344.991 86.33,342.225 86.831,339.753 87.332,339.495 87.833,335.509 88.334,331.837 88.835,330.777 
 89.336,331.447 89.837,331.227 90.338,330.328 90.839,333.783 91.34,332.682 91.841,331.445 92.342,338.745 92.843,339.262 93.344,338.939 93.845,337.616 
 94.346,337.45 94.847,337.6 95.348,337.117 95.849,332.626 96.35,333.276 96.851,330.822 97.352,332.766 97.853,332.002 98.354,332.378 98.855,331.735 
 99.356,331.926 99.857,333.262 100.358,333.67 100.859,333.286 101.36,333.252 101.861,326.587 102.362,326.756 102.863,329.057 103.364,330.244 103.865,324.805 
 104.366,323.812 104.867,319.474 105.368,320.906 105.869,319.925 106.37,319.256 106.871,314.87 107.372,308.969 107.873,306.762 108.374,307.933 108.875,309.42 
 109.376,313.331 109.877,315.69 110.378,310.619 110.879,323.468 111.38,323.366 111.881,320.217 112.382,316.15 112.883,314.876 113.384,309.421 113.885,308.156 
 114.386,308.603 114.887,309.683 115.388,308.987 115.889,308.967 116.39,310.049 116.891,310.926 117.392,308.292 117.893,279.783 118.394,283.493 118.895,278.774 
 119.396,276.001 119.897,277.099 120.398,277.178 120.899,276.524 121.4,276.492 121.901,277.843 122.402,283.988 122.903,284.399 123.404,284.003 123.905,283.133 
 124.406,284.36 124.907,284.323 125.408,284.863 125.909,284.8 126.41,285.692 126.911,285.14 127.412,284.577 127.913,283.887 128.414,283.314 128.915,282.669 
 129.416,282.421 129.917,282.494 130.418,282.184 130.919,283.588 131.42,283.778 131.921,284.242 132.422,285.132 132.923,284.528 133.424,286.667 133.925,287.013 
 134.426,286.653 134.927,282.405 135.428,279.958 135.929,280.743 136.43,281.105 136.931,282.334 137.432,278.341 137.933,278.54 138.434,278.506 138.935,286.463 
 139.436,285.245 139.937,284.432 140.438,287.02 140.939,285.045 141.44,286.627 141.941,284.409 142.442,286.14 142.943,291.059 143.444,290.57 143.945,291.934 
 144.446,291.916 144.947,288.164 145.448,294.339 145.949,299.513 146.45,299.162 146.951,299.304 147.452,301.134 147.953,302.33 148.454,306.484 148.955,307.153 
 149.456,307.475 149.957,306.032 150.458,309.049 150.959,308.398 151.46,307.509 151.961,309.214 152.462,309.015 152.963,308.249 153.464,307.074 153.965,309.379 
 154.466,311.009 154.967,314.476 155.468,314.756 155.969,315.181 156.47,309.816 156.971,311.149 157.472,310.548 157.973,310.765 158.474,308.84 158.975,310.029 
 159.476,310.515 159.977,312.819 160.478,313.549 160.979,316.437 161.48,321.616 161.981,323.433 162.482,325.392 162.983,328.24 163.484,328.548 163.985,330.591 
 164.486,331.947 164.987,335.462 165.488,335.691 165.989,335.001 166.49,336.658 166.991,339.802 167.492,341.265 167.993,345.655 168.494,344.762 168.995,346.197 
 169.496,346.906 169.997,346.819 170.498,368.831 170.999,374.677 171.5,375.455 172.001,375.55 172.502,375.767 173.003,375.776 173.504,375.771 174.005,375.767 
 174.506,375.775 175.007,375.783 175.508,375.786 176.009,375.786 176.51,375.784 177.011,375.659 177.512,375.661 178.013,375.653 178.514,375.64 179.015,375.633 
 179.516,375.634 180.017,375.65 180.518,375.644 181.019,375.655 181.52,375.693 182.021,375.721 182.522,375.707 183.023,375.708 183.524,375.703 184.025,375.789 
 184.526,375.791 185.027,375.737 185.528,375.8 186.029,375.8 186.53,375.801 187.031,375.802 187.532,375.803 188.033,375.737 188.534,375.799 189.035,375.798 
 189.536,375.796 190.037,375.798 190.538,375.798 191.039,375.797 191.54,375.794 192.041,375.796 192.542,375.724 193.043,375.726 193.544,375.73 194.045,375.724 
 194.546,375.796 195.047,375.718 195.548,375.716 196.049,375.722 196.55,375.793 197.051,375.791 197.552,375.796 1